In [ ]:
from mesa import Agent, Model
from mesa.space import NetworkGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
import networkx as nx
import graph_init
import pandas as pd
import Station as st

In [ ]:
class Commuter(Agent):
    def __init__(self, model, distance_left, destination):
        super().__init__(model)
        self.destination = destination
        self.distance_left = distance_left
        self.current_pos = self.model.get_node(self)
        
    def step(self):
        self.current_pos = self.model.get_node(self)  # Update the agent's current position
        if self.distance_left==0:
            # move the agent to the destination
            self.model.grid.move_agent(self, self.destination)
            self.current_pos = self.model.get_node(self)
            self.destination = self.model.random.choice([n for n in self.model.grid.G.nodes() if n != self.current_pos])
            self.distance_left = len(self.model.path_finder(self.current_pos, self.destination))
        self.distance_left -= 1  # Update the distance left to the destination
    
    def get_agent_position(self):
    # needed seperate function for data collector
        return self.current_pos
    
    def get_distance_left(self):
    # needed seperate function for data collector
        return self.distance_left
    
    def get_agent_destination(self):
        return self.destination


class MyModel(Model):
    def __init__(self, n_agents, nodes=10, max_distance=5):
        super().__init__()
        self.grid = NetworkGrid(graph_init.create_random_graph(nodes,max_distance))
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            agent_reporters={"Position": lambda agent: agent.get_agent_position(),
                             "Distance_Left": lambda agent: agent.get_distance_left(),
                             "Destination": lambda agent: agent.get_agent_destination()})
        

        # Create agents and place them on the grid
        for i in range(1, n_agents + 1):
            node_id = self.random.choice(list(self.grid.G.nodes()))
            destination_node = self.random.choice([n for n in self.grid.G.nodes() if n != node_id])
            distance_left = len(self.path_finder(node_id, destination_node))
            commuter = Commuter(self, distance_left=distance_left, destination=destination_node)
            self.schedule.add(commuter)
            self.grid.place_agent(commuter, node_id)
            commuter.current_pos = node_id
            # print(f"get_node_test: Agent {commuter.unique_id} placed at node {self.get_node(commuter)}")
            # print(f"Agent {commuter.unique_id} placed at node {node_id}")
        self.datacollector.collect(self)
        
        # Create stations and place them on the grid
        for i in range(nodes):
            self.grid.G.nodes[i]['station'] = st.Station(capacity=5, available_bikes=3)

    def path_finder(self, start, end):
        return nx.shortest_path(self.grid.G, source=start, target=end, weight='weight')
    
    def get_node(self, agent):
        # Iterate over all node IDs in the graph
        for node_id in self.grid.G.nodes():
            # Get the agents at the current node
            agents_at_node = self.grid.get_cell_list_contents([node_id])
            # If the given agent is found at this node, return the node ID
            if agent in agents_at_node:
                return node_id
        return None  # If the agent is not found in any node



    def step(self):
        # print("Model step")
        self.schedule.step()
        self.datacollector.collect(self)

In [ ]:
model = MyModel(20)
for i in range(5):
    model.step()
agent_data = model.datacollector.get_agent_vars_dataframe()
agent_id = 8
agent_specific_data = agent_data[agent_data.index.get_level_values('AgentID') == agent_id]

# Print the filtered DataFrame
print(agent_specific_data)